## Python 타입 힌트
### `Dict`와 `TypedDict` 비교
| 항목           | `Dict`                                    | `TypedDict`                                     |
| ------------ | ------------------------------------------ | ----------------------------------------------- |
| **정의 위치**    | `typing.Dict` 또는 `dict` (권장)                   | `typing.TypedDict`                              |
| **타입 검사 시점** | 런타임만 (정적 타입 검사 불가능)                        | 정적 타입 검사 가능 (IDE, mypy 등)                       |
| **구조 정의**    | 키/값 타입만 지정 가능 (`Dict[str, int]`)           | 각 키에 대해 이름별로 타입 정의 가능 (`name: str, age: int` 등) |
| **자동완성 지원**  | 없음                                         | IDE 자동완성 가능 (`person["name"]` 등에서 지원됨)          |
| **유연성**      | 아무 키나 추가/삭제 가능                             | 타입 검사시 정의된 키 외 사용 시 경고, 런타임에는 일반 dict처럼 작동     |
| **가독성/문서화**  | 구조 추론 어려움                                  | 구조 명시로 가독성 및 문서화 효과 높음                          |
| **런타임 성능**   | 빠름 (기본 내장형)                                | dict 기반이므로 성능 유사                                |

* 정적검사에서 오류가 발생해도 런타임에는 값이 입력됨

In [ ]:
# 정적 검사를 위한 패키지 설치(pip install nb-mypy)
# nb-mypy 확장 모듈을 로딩

%load_ext nb_mypy

In [ ]:
from typing import Dict, TypedDict

# Dict 기반 정의: 자유롭게 key-value를 저장 (타입 검사 없음)
sample_dict: Dict[str, str] = {
    "name": "에도가와 코난",
    "age": "17",
    "job": "고등학생 탐정",
}

# TypedDict 기반 정의: 타입 검사 지원 (정적 검사 도구에서 검출 가능)
class Detective(TypedDict):
    name: str
    age: int
    job: str

# 오류 유도 예제: "address"는 정의되어 있지 않음
typed_dict: Detective = {
    "name": "에도가와 코난",
    "age": 17,
    "job": "고등학생 탐정",
    "address": "도쿄 비이카초",  # 정의되지 않은 필드 → 정적 검사 시 오류 발생
}

In [ ]:
# Dict : 정적 검사 age에 숫자
sample_dict["age"] = 28
sample_dict["new_field"] = "새로운 정보를 추가 합니다"

# TypedDict: 정적 검사 "new_field"
typed_dict["age"] = 28
typed_dict["new_field"] = "새로운 정보를 추가 합니다"

print(sample_dict)
print(typed_dict)

### 타입 힌트 메타 데이터 `Annotated`

*`Annotated`는 타입 힌트에 추가적인 메타데이터를 첨부할 수 있게 해주는 기능 (Pydantic v2 권장 방식)

```
Annotated[타입, 메타데이터1, 메타데이터2, ...]
```
* `Field`는 유효성 검사 조건, 기본값, 메타데이터 (예: 설명, 예시) 등을 추가하기 위해 주로 Annotated 또는 직접 필드 기본값 자리에 사용

| `Field` 주요 매개변수          | 설명                                        |
| ------------- | ----------------------------------------- |
| `...`     | 필수 필드(`required`)                     |
| `default`     | 기본값 (생략하면 필수 필드로 간주됨)                     |
| `gt`, `ge`    | greater than / greater or equal (>) / (≥) |
| `lt`, `le`    | less than / less or equal (<) / (≤)       |
| `min_length`  | 문자열 최소 길이                                 |
| `max_length`  | 문자열 최대 길이                                 |
| `description` | 설명 (문서화용)                                 |
| `example`     | 예시 값 (문서화용)                               |
| `title`       | 제목 (문서화용)                                 |
| `deprecated`  | 더 이상 사용되지 않음을 명시                          |

---
```python
# 예시 (기존 방식)
from pydantic import BaseModel, Field

class User(BaseModel):
    name: str = Field(..., title="이름", description="사용자의 실명", max_length=20)
    age: int = Field(gt=0, le=120, example=25)
    nickname: str = Field(default="guest", deprecated=True)

/**
- `name`: 필수 입력이며 20자 이하
- `age`: 0보다 크고 120 이하
- `nickname`: 기본값 "guest", 더 이상 권장되지 않음
*/
```


In [ ]:
from typing import Annotated, List
from pydantic import BaseModel, Field, ValidationError

class DetectiveCharacter(BaseModel):
    codename: Annotated[str, Field(..., description="캐릭터 코드네임 (예: 코난, 키드)")]
    real_name: Annotated[str, Field(..., min_length=2, max_length=50, description="실명")]
    age: Annotated[int, Field(gt=6, lt=21, description="나이 (7~20세 사이의 범위로 제한)")]
    roles: Annotated[
        List[str], Field(min_items=1, max_items=5, description="역할(1~5개): 탐정, 조수, 범인 등")
    ]

In [ ]:
try:
    valid_character = DetectiveCharacter(
        codename="코난",
        real_name="쿠도 신이치",
        age=17,
        roles=["탐정", "고등학생"],
    )
    print("캐릭터 적합: ", valid_character)

except ValidationError as e:
    print("캐릭터 부적합: ", e)

valid_character

In [ ]:
# 유효하지 않은 케이스로 시도
try:
    invalid_character = DetectiveCharacter(
        codename="버본",
        real_name="아무로 토오루",
        age=35,  # 나이 범위 초과
        roles=["FBI", "요원"]
    )
    print("캐릭터 적합: ", invalid_character)
except ValidationError as e:
    print("not_relevant: ", e)

invalid_character

-----
** End of Documents **